In [1]:
import re
import pandas as pd
import zhconv #繁体字转换
import tensorflow as tf
# tf.keras.layers.experimental.preprocessing.TextVectorization

In [ ]:
def load_data(data_file):
    lines = list(open(data_file, 'r', encoding='utf-8').readlines())
    y = [line[:1] for line in lines]
    x = [clean_str(line[1:] for lin in lines)]
    return [x, y]

def cleaan_str(string):
    string = re.sub(u'[\u4e00-\u9fa5]+', ' ', string)
    string = zhconv.convert(string.strip(), 'zh-hans')

    return string


load_data(data_file='../data/文本分类/sms_pub.csv')


In [54]:
BUFFER_SIZE = 10000
BATCH_SIZE = 1


dataset = tf.data.experimental.make_csv_dataset(
  file_pattern='../data/文本分类/sms_pub.csv',
  field_delim=',',
  batch_size=BATCH_SIZE,
  label_name="label",
  select_columns=['message', 'label'],
  shuffle=True
  )#.map(split_line, num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# print(next(iter(dataset)), '\n', dataset.take(1))
print(type(dataset))
for feature_batch, label_batch in dataset.take(1):
  print(len(label_batch), len(feature_batch['message']))
  for i in range(1):
    print(label_batch[i], '\n', feature_batch['message'][i][:1])

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
1 1
tf.Tensor([0], shape=(1,), dtype=int32) 
 tf.Tensor([b'\xe5\x88\x9a\xe6\x89\x8d\xe2\x80\xa6\xe6\x88\x91\xe5\x9c\xa8\xe5\x9c\xb0\xe9\x93\x81\xe4\xb8\x8a\xe2\x80\xa6\xe8\xa2\xab\xe8\xae\xa9\xe5\xba\xa7\xe4\xba\x86\xe2\x80\xa6\xe8\xbf\x99\xe2\x80\xa6\xe6\x88\x91\xe4\xbb\xa5\xe4\xb8\xba\xe4\xbb\x96\xe6\x98\xaf\xe4\xb8\x8b\xe4\xb8\x80\xe7\xab\x99\xe4\xb8\x8b\xe8\xbd\xa6'], shape=(1,), dtype=string)


In [58]:
MAX_WORDS = 10000  # 仅考虑最高频的10000个词
MAX_LEN = 200  # 每个样本保留200个词的长度
BATCH_SIZE = 20


#构建词典
def clean_text(text):
    lowercase = tf.strings.lower(text)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_punctuation = tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation),'')
    return cleaned_punctuation

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    standardize = clean_text,
    split = 'whitespace',
    max_tokens = MAX_WORDS-1, #有一个留给占位符
    output_mode = 'int',
    output_sequence_length = MAX_LEN)


vectorize_layer.adapt(dataset.map(lambda text, label: text))
print(vectorize_layer.get_vocabulary()[0:100])

encoder.get_vocabulary()[:20]

AttributeError: in user code:

    File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/keras/engine/base_preprocessing_layer.py", line 117, in adapt_step  *
        self._adapt_maybe_build(data)
    File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/keras/engine/base_preprocessing_layer.py", line 280, in _adapt_maybe_build  **
        self.build(data_shape)
    File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/keras/layers/preprocessing/text_vectorization.py", line 445, in build
        if input_shape.ndims > 1 and not input_shape[-1] == 1:  # pylint: disable=g-comparison-negation

    AttributeError: 'NoneType' object has no attribute 'ndims'


In [ ]:
batch_size = 32
seed = 42


ds_data = tf.keras.utils.text_dataset_from_directory(
    directory=['../data/文本分类'],
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

ds_data

In [ ]:
categorical_columns = []


for element in dataset.as_numpy_iterator():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=str(element['label'][0]),
        vocabulary_list=element['message'],
        num_oov_buckets=2
        )
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    break

categorical_columns

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns)
preprocessing_layer

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
dataset

In [ ]:
model.fit(dataset.shuffle(500), epochs=20)